In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math 
import os

from opencage.geocoder import OpenCageGeocode

In [43]:
# Import xlsx file and store each sheet in to a df list
xl_file = pd.ExcelFile('./data.xls',)

dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

# Data from each sheet can be accessed via key
keyList = list(dfs.keys())

In [44]:
# Save each sheet as csv to improve performance (these csv files will be used for app.py)
for key, df in dfs.items():
    dfs[key].to_csv('./raw_data/{}.csv'.format(key), index = False)

In [45]:
# Data cleansing
for key, df in dfs.items():
    dfs[key].loc[:,'Confirmed'].fillna(value=0, inplace=True)
    dfs[key].loc[:,'Deaths'].fillna(value=0, inplace=True)
    dfs[key].loc[:,'Recovered'].fillna(value=0, inplace=True)
    dfs[key]=dfs[key].astype({'Confirmed':'int64', 'Deaths':'int64', 'Recovered':'int64'})
    # Change as China for coordinate search
    dfs[key]=dfs[key].replace({'Country/Region':'Mainland China'}, 'China')
    # Add a zero to the date so can be convert by datetime.strptime as 0-padded date
    dfs[key]['Last Update'] = '0' + dfs[key]['Last Update']
    # Convert time as Australian eastern daylight time
    dfs[key]['Date_last_updated_AEDT'] = [datetime.strptime(d, '%m/%d/%Y %H:%M') for d in dfs[key]['Last Update']]
    dfs[key]['Date_last_updated_AEDT'] = dfs[key]['Date_last_updated_AEDT'] + timedelta(hours=16)

### Code of the following cell is for generating cumulative data for lineplot. They should run when updating heroku server folder.

In [46]:
if not os.path.exists('./cumulative_data'):
    os.makedirs('./cumulative_data')

# As countries (China, US, Canada and Australia) have duplicates as these counties have Province/State data
for Region in set(dfs[keyList[0]]['Country/Region']): 
    # Function for generating cumulative line plot for each Country/Region
    CaseType = ['Confirmed', 'Recovered', 'Deaths']

    # Construct confirmed cases dataframe for line plot
    df_region = pd.DataFrame(columns=['Confirmed', 'Recovered', 'Deaths', 'Date_last_updated_AEDT'])

    for key, df in dfs.items():
        # As Country name will not be in the dataframe when there is no cases
        if Region in list(df['Country/Region']):
            dfTpm = df.groupby(['Country/Region']).agg({'Confirmed':np.sum, 
                                                        'Recovered':np.sum, 
                                                        'Deaths':np.sum, 
                                                        'Date_last_updated_AEDT':'first'})
            df_region = df_region.append(dfTpm.loc[Region, :])        
        else:
            dfTpm2 = pd.DataFrame({'Confirmed':[0],
                                   'Recovered':[0],
                                   'Deaths':[0],
                                   'Date_last_updated_AEDT':[df['Date_last_updated_AEDT'][0]]}, index=[Region])
            df_region = df_region.append(dfTpm2)

    # Select the latest data from a given date
    df_region['date_day']=[d.date() for d in df_region['Date_last_updated_AEDT']]
    df_region=df_region.groupby(by=df_region['date_day'], sort=False).first()

    df_region=df_region.reset_index(drop=True)
    df_region['New'] = -(df_region['Confirmed'].shift(-1) - df_region['Confirmed'])
    df_region.at[df_region.shape[0]-1, 'New'] = 0
    df_region = df_region.astype({'New':int})
    df_region.to_csv('./cumulative_data/{}.csv'.format(Region), index = False)

print('Cumulative data of each region are generated!')

Cumulative data of each region are generated!


### This part is for saving all coordinates as my own database. By doing so, `opencage.geocoder` does not need to go through all regions everytime (as most regions are already have coordinates in this database). Only new added regions will be called for coordinates via `opencage.geocoder`.

In [47]:
def coordinateCalling(queryData: pd.DataFrame) -> pd.DataFrame:
    '''
    Using opencage.geocoder to call coordinates for these regions
    Add coordinates for each region in the list for the latest table sheet
    '''
    key = 'b33700b33d0a446aa6e16c0b57fc82d1'  # get api key from:  https://opencagedata.com
    geocoder = OpenCageGeocode(key)

    list_lat = []   # create empty lists
    list_long = []  

    for index, row in queryData.iterrows(): # iterate over rows in dataframe

        City = row['Province/State']
        State = row['Country/Region']

        # Note that 'nan' is float
        if type(City) is str:
            query = str(City)+','+str(State)
            results = geocoder.geocode(query)   
            lat = results[0]['geometry']['lat']
            long = results[0]['geometry']['lng']

            list_lat.append(lat)
            list_long.append(long)
        else:
            query = str(State)
            results = geocoder.geocode(query)   
            lat = results[0]['geometry']['lat']
            long = results[0]['geometry']['lng']

            list_lat.append(lat)
            list_long.append(long)

    # create new columns from lists    
    queryData['lat'] = list_lat   
    queryData['lon'] = list_long
    
    return queryData   

In [48]:
# Import coordinate database
GeoDB = pd.read_csv('./coordinatesDB.csv')

# Save the latest data into targetData
targetData = dfs[keyList[0]]

# Assign coordinates to regions from coordinates database
resultData = pd.merge(targetData, GeoDB, how='left', on=['Province/State', 'Country/Region'])

# Find regions do not have coordinates
queryData = resultData.loc[resultData['lat'].isnull()]
queryData = queryData[['Province/State', 'Country/Region']]

# There will be two possible results
# One queryData is empty when all regions already have coordinates from coordinate database,
# and queryData is not empty when there are new added region names
if queryData.shape[0] != 0:
    coordinateCalling(queryData)
    # Update database by adding the new coordinates into it
    GeoDB = pd.concat([GeoDB, queryData], ignore_index=True)
    # Save the coordinates database
    GeoDB.to_csv('./coordinatesDB.csv', index = False)
    # Assign coordinates to all regions using the latest coordinates database
    finalData = pd.merge(targetData, GeoDB, how='left', on=['Province/State', 'Country/Region'] )
    
    # To check if there is still regions without coordinates (There should not be)
    if finalData.loc[finalData['lat'].isnull()].shape[0] == 0:
        # Save the data for heroku app
        finalData.to_csv('./{}_data.csv'.format(keyList[0]), index = False)
        print('New regions now have corordinates. Data has been saved.')
    else:
        # There might be some issues
        print('Please check your data') 
else:
    # Assign coordinates to all regions using the latest coordinates database
    finalData = pd.merge(targetData, GeoDB, how='left', on=['Province/State', 'Country/Region'] )
    # Save the data for heroku app
    finalData.to_csv('./{}_data.csv'.format(keyList[0]), index = False)
    print('There is no added region. Data has been saved')
 

There is no added region. Data has been saved


In [49]:
queryData

,Province/State,Country/Region


### Save all csv files and copy to heroku folder

In [50]:
# A variable for using in bash 
# Refer to https://stackoverflow.com/questions/19579546/can-i-access-python-variables-within-a-bash-or-script-ipython-notebook-c
fileName1 = keyList[0]
fileName2 = keyList[1]

In [51]:
%%bash -s "$fileName1" "$fileName2"
# Delete previous data from heroku folder
rm ../../heroku_app/dash_coronavirus_2019/$2_data.csv
# Delete previous data from current folder
rm ./$2_data.csv
# Copy all required data to heroku folder
cp ./data.xls ../../heroku_app/dash_coronavirus_2019/
cp ./raw_data/*.csv ../../heroku_app/dash_coronavirus_2019/raw_data/
cp ./cumulative_data/*.csv ../../heroku_app/dash_coronavirus_2019/cumulative_data/
cp ./$1_data.csv ../../heroku_app/dash_coronavirus_2019/
echo "All files have been transferred to heroku folder."
echo "You are now good to update heroku app!"

All files have been transferred to heroku folder.
You are now good to update heroku app!
